# SQL

# Задание 1

### Условие

Образовательные курсы состоят из различных уроков, каждый из которых состоит из нескольких маленьких заданий. Каждое такое маленькое задание называется "горошиной".

Назовём очень усердным учеником того пользователя, который хотя бы раз за текущий месяц правильно решил 20 горошин.

In [12]:
import pandahouse as ph
from datetime import datetime

In [13]:
# Подключаемся к базе данных

connection_default = {'host': 'https://clickhouse.lab.karpov.courses',
                      'database':'default',
                      'user':'student', 
                      'password':'dpo_python_2020'
                     }

Посмотрим на данные

In [14]:
query = '''
    SELECT
        st_id,
        timest::DATETIME AS timest,
        correct,
        subject        
    FROM default.peas
    LIMIT 3
    '''

peas = ph.read_clickhouse(query=query, connection=connection_default)
peas

,st_id,timest,correct,subject
0,100379,2021-10-30 13:32:29,1,Theory of probability
1,100379,2021-10-30 14:11:19,0,Vizualization
2,100379,2021-10-30 15:54:22,1,Theory of probability


In [15]:
query = '''
    SELECT
        st_id,
        sale_time::DATETIME AS sale_time,
        money,
        subject        
    FROM default.final_project_check
    LIMIT 3
    '''

final_project_check = ph.read_clickhouse(query=query, connection=connection_default)
final_project_check

,st_id,sale_time,money,subject
0,101432,2021-10-31 04:44:32,85000,Math
1,101432,2021-10-31 12:43:50,65000,Vizualization
2,104885,2021-10-30 17:05:55,65000,Vizualization


In [16]:
query = '''
    SELECT
        st_id,
        test_grp       
    FROM default.studs
    LIMIT 3
    '''

studs = ph.read_clickhouse(query=query, connection=connection_default)
studs

,st_id,test_grp
0,100379,pilot
1,101432,control
2,104818,pilot


Посмотрим на период данных

In [17]:
query = '''
    SELECT
        MIN(timest::DATE) as min,
        MAX(timest::DATE) as max
    FROM default.peas
'''

date_interval = ph.read_clickhouse(query=query, connection=connection_default)
date_interval

,min,max
0,2021-10-30,2021-10-31


In [18]:
query = '''
    SELECT
        MIN(sale_time::DATE) as min,
        MAX(sale_time::DATE) as max
    FROM default.final_project_check
'''

date_interval_s = ph.read_clickhouse(query=query, connection=connection_default)
date_interval_s

,min,max
0,2021-10-30,2021-10-31


In [19]:
# Посчитаем количество усердных студентов

query = '''
    SELECT
        COUNT(*) as diligent_student
    FROM
    (
        SELECT
            SUM(correct) as solved_problems
        FROM 
            default.peas
        GROUP BY st_id
    )
    WHERE solved_problems >= 20
    '''

diligent_student = ph.read_clickhouse(query, connection=connection_default)
diligent_student

,diligent_student
0,136


Ответ: 136 усердных студентов

# Задание 2

### Оптимизация воронки

### Условие



Образовательная платформа предлагает пройти студентам курсы по модели trial: студент может решить бесплатно лишь 30 горошин в день. Для неограниченного количества заданий в определенной дисциплине студенту необходимо приобрести полный доступ. Команда провела эксперимент, где был протестирован новый экран оплаты.

In [20]:
query = '''
    WITH active_users AS (
        SELECT st_id
        FROM default.peas
        GROUP BY st_id
        HAVING COUNT(CASE WHEN correct = true THEN 1 END) > 10
    ),
    math_active_users AS (
        SELECT st_id
        FROM default.peas 
        WHERE subject = 'Math'
        GROUP BY st_id
        HAVING COUNT(CASE WHEN correct = true THEN 1 END) >= 2
    )

    SELECT
        s.test_grp AS test_grp,
        ROUND(SUM(fpc.money) / COUNT(DISTINCT s.st_id), 2) AS ARPU,
        ROUND(SUM(fpc.money) FILTER(WHERE s.st_id = active_users.st_id)  / COUNT(DISTINCT s.st_id) FILTER(WHERE s.st_id = active_users.st_id), 2) AS ARPAU,
        ROUND(COUNT(DISTINCT s.st_id) FILTER(WHERE fpc.money > 0) / COUNT(DISTINCT s.st_id) * 100, 2) AS CR,
        ROUND(COUNT(DISTINCT s.st_id) FILTER(WHERE fpc.money > 0 and s.st_id = active_users.st_id) / COUNT(DISTINCT s.st_id) FILTER(WHERE s.st_id = active_users.st_id) * 100, 2) AS CR_active_users,
        ROUND(COUNT(DISTINCT s.st_id) FILTER(WHERE fpc.money > 0 and s.st_id = math_active_users.st_id) / COUNT(DISTINCT s.st_id) FILTER(WHERE s.st_id = math_active_users.st_id) * 100, 2) AS CR_math_active_users
    FROM
        default.studs AS s
    LEFT JOIN
        default.final_project_check AS fpc ON s.st_id = fpc.st_id
    LEFT JOIN
        active_users ON s.st_id = active_users.st_id
    LEFT JOIN
        math_active_users ON s.st_id = math_active_users.st_id
    GROUP BY
        s.test_grp
'''

metric_task = ph.read_clickhouse(query, connection=connection_default)

In [21]:
metric_task

,test_grp,ARPU,ARPAU,CR,CR_active_users,CR_math_active_users
0,control,4540.98,10393.70,4.92,11.02,10.20
1,pilot,11508.47,29739.58,10.85,26.04,14.29


In [22]:
print('По сравнению с контрольной группой все метрики в тестовой выросли')
print(f'''ARPU увеличился на {round(metric_task.query('test_grp == "pilot"').ARPU[1] / metric_task.query('test_grp == "control"').ARPU[0], 2)}%''')
print(f'''ARPAU увеличился на {round(metric_task.query('test_grp == "pilot"').ARPAU[1] / metric_task.query('test_grp == "control"').ARPAU[0], 2)}%''')
print(f'''CR в покупку увеличился на {round(metric_task.query('test_grp == "pilot"').CR[1] / metric_task.query('test_grp == "control"').CR[0], 2)}%''')
print(f'''СR активного пользователя в покупку увеличился на {round(metric_task.query('test_grp == "pilot"').CR_active_users[1] / metric_task.query('test_grp == "control"').CR_active_users[0], 2)}%''')
print(f'''CR пользователя из активности по математике в покупку курса по математике увеличился на {round(metric_task.query('test_grp == "pilot"').CR_math_active_users[1] / metric_task.query('test_grp == "control"').CR_math_active_users[0], 2)}%''')

По сравнению с контрольной группой все метрики в тестовой выросли
ARPU увеличился на 2.53%
ARPAU увеличился на 2.86%
CR в покупку увеличился на 2.21%
СR активного пользователя в покупку увеличился на 2.36%
CR пользователя из активности по математике в покупку курса по математике увеличился на 1.4%
